In [1]:
# 从sklearn 调入所需要的包
# from sklearn import datasets
from sklearn.model_selection import train_test_split #数据分隔出训练集和验证集
import lightgbm as lgb
import numpy as np 
import pandas as pd
#导入精度和召回
from sklearn.metrics import precision_score, recall_score
import lightgbm as lgb

In [3]:
path='./'
train = pd.read_csv(path+'input/train.csv')
test = pd.read_csv(path+'input/test_noLabel.csv')
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 32 columns):
ID                          1100 non-null int64
Age                         1100 non-null int64
BusinessTravel              1100 non-null object
Department                  1100 non-null object
DistanceFromHome            1100 non-null int64
Education                   1100 non-null int64
EducationField              1100 non-null object
EmployeeNumber              1100 non-null int64
EnvironmentSatisfaction     1100 non-null int64
Gender                      1100 non-null object
JobInvolvement              1100 non-null int64
JobLevel                    1100 non-null int64
JobRole                     1100 non-null object
JobSatisfaction             1100 non-null int64
MaritalStatus               1100 non-null object
MonthlyIncome               1100 non-null int64
NumCompaniesWorked          1100 non-null int64
Over18                      1100 non-null object
OverTime              

In [4]:
train.describe()

,ID,Age,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Label
count,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,...,1100.0,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000
mean,549.500000,36.999091,9.427273,2.922727,1028.157273,2.725455,2.730909,2.054545,2.732727,6483.620909,...,80.0,0.788182,11.221818,2.807273,2.746364,7.011818,4.207273,2.226364,4.123636,0.161818
std,317.686953,9.037230,8.196694,1.022242,598.915204,1.098053,0.706366,1.107805,1.109731,4715.293419,...,0.0,0.843347,7.825548,1.291514,0.701121,6.223093,3.618115,3.313830,3.597996,0.368451
min,0.000000,18.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1009.000000,...,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,274.750000,30.000000,2.000000,2.000000,504.250000,2.000000,2.000000,1.000000,2.000000,2924.500000,...,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000,0.000000
50%,549.500000,36.000000,7.000000,3.000000,1026.500000,3.000000,3.000000,2.000000,3.000000,4857.000000,...,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000,0.000000
75%,824.250000,43.000000,15.000000,4.000000,1556.500000,4.000000,3.000000,3.000000,4.000000,8354.500000,...,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000,0.000000
max,1099.000000,60.000000,29.000000,5.000000,2065.000000,4.000000,4.000000,5.000000,4.000000,19999.000000,...,80.0,3.000000,40.000000,6.000000,4.000000,37.000000,18.000000,15.000000,17.000000,1.000000


In [5]:
test.describe()

,ID,Age,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,...,350.000000,350.0,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000
mean,1274.500000,36.471429,8.391429,2.868571,1023.285714,2.714286,2.734286,2.068571,2.725714,6479.491429,...,2.745714,80.0,0.817143,11.202857,2.782857,2.808571,6.782857,4.260000,1.951429,4.017143
std,101.180532,9.373378,7.685318,1.029583,612.566819,1.067129,0.726669,1.089615,1.083437,4633.609813,...,1.041226,0.0,0.886539,7.470399,1.295238,0.722488,5.489113,3.622336,2.752532,3.383720
min,1100.000000,18.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1051.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1187.250000,30.000000,2.000000,2.000000,463.250000,2.000000,2.000000,1.000000,2.000000,2888.750000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,1274.500000,35.000000,7.000000,3.000000,1011.000000,3.000000,3.000000,2.000000,3.000000,5104.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,1361.750000,42.000000,11.000000,4.000000,1584.500000,4.000000,3.000000,3.000000,4.000000,8260.250000,...,4.000000,80.0,1.000000,15.750000,3.000000,3.000000,9.750000,7.000000,2.000000,7.000000
max,1449.000000,60.000000,29.000000,5.000000,2068.000000,4.000000,4.000000,5.000000,4.000000,19973.000000,...,4.000000,80.0,3.000000,37.000000,6.000000,4.000000,29.000000,16.000000,15.000000,14.000000


In [6]:
train.corr()

,ID,Age,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Label
ID,1.000000,0.002382,0.037403,-0.021587,-0.014292,0.001818,0.062834,-0.036465,-0.027260,-0.027679,...,NaN,-0.000762,-0.023704,-0.021694,0.006935,-0.038080,-0.024975,-0.016626,-0.023698,0.018291
Age,0.002382,1.000000,0.007081,0.198558,-0.010953,0.011803,0.066528,0.513882,-0.003744,0.500163,...,NaN,-0.002413,0.682879,-0.051702,-0.001042,0.328651,0.231842,0.230587,0.212540,-0.175393
DistanceFromHome,0.037403,0.007081,1.000000,0.011437,0.029930,-0.010308,0.012333,0.016470,-0.009641,-0.017757,...,NaN,0.050356,0.001287,-0.041208,-0.050950,0.000044,0.019317,-0.002760,0.008852,0.088563
Education,-0.021587,0.198558,0.011437,1.000000,0.055979,-0.032698,0.022843,0.084075,-0.010201,0.079834,...,NaN,0.035881,0.125672,-0.021629,0.003099,0.074522,0.064363,0.067754,0.071870,-0.046494
EmployeeNumber,-0.014292,-0.010953,0.029930,0.055979,1.000000,0.030428,-0.008589,-0.006121,-0.042443,-0.007147,...,NaN,0.049967,-0.009526,0.017175,0.000548,-0.011106,-0.005649,-0.019621,-0.005333,-0.045168
EnvironmentSatisfaction,0.001818,0.011803,-0.010308,-0.032698,0.030428,1.000000,-0.028467,-0.015355,0.000212,-0.026410,...,NaN,0.008874,-0.018532,-0.045686,0.026477,-0.012574,0.003572,0.008843,-0.020190,-0.097003
JobInvolvement,0.062834,0.066528,0.012333,0.022843,-0.008589,-0.028467,1.000000,0.005983,-0.016382,0.006114,...,NaN,0.029483,0.018380,-0.018001,-0.025862,-0.032189,0.001194,-0.031097,0.014176,-0.122722
JobLevel,-0.036465,0.513882,0.016470,0.084075,-0.006121,-0.015355,0.005983,1.000000,-0.005894,0.950776,...,NaN,0.002638,0.784020,-0.034620,0.041258,0.544091,0.411481,0.395195,0.376119,-0.168775
JobSatisfaction,-0.027260,-0.003744,-0.009641,-0.010201,-0.042443,0.000212,-0.016382,-0.005894,1.000000,-0.009752,...,NaN,0.021123,-0.023343,0.002754,-0.042767,-0.013772,-0.011798,-0.009761,-0.041852,-0.125568
MonthlyIncome,-0.027679,0.500163,-0.017757,0.079834,-0.007147,-0.026410,0.006114,0.950776,-0.009752,1.000000,...,NaN,-0.006320,0.772043,-0.039713,0.032247,0.520641,0.388163,0.386875,0.344237,-0.155521


In [7]:
train.head()

,ID,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Label
0,0,37,Travel_Rarely,Research & Development,1,4,Life Sciences,77,1,Male,...,80,1,7,2,4,7,5,0,7,0
1,1,54,Travel_Frequently,Research & Development,1,4,Life Sciences,1245,4,Female,...,80,1,33,2,1,5,4,1,4,0
2,2,34,Travel_Frequently,Research & Development,7,3,Life Sciences,147,1,Male,...,80,0,9,3,3,9,7,0,6,1
3,3,39,Travel_Rarely,Research & Development,1,1,Life Sciences,1026,4,Female,...,80,1,21,3,3,21,6,11,8,0
4,4,28,Travel_Frequently,Research & Development,1,3,Medical,1111,1,Male,...,80,2,1,2,3,1,0,0,0,1


In [8]:
test.head()

,ID,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1100,40,Non-Travel,Research & Development,9,4,Other,1449,3,Male,...,3,80,2,11,2,4,8,7,0,7
1,1101,53,Travel_Rarely,Research & Development,7,2,Medical,1201,4,Female,...,2,80,1,26,6,3,7,7,4,7
2,1102,42,Travel_Rarely,Research & Development,2,4,Other,477,1,Male,...,2,80,0,14,6,3,1,0,0,0
3,1103,34,Travel_Frequently,Human Resources,11,3,Life Sciences,1289,3,Male,...,4,80,2,14,5,4,10,9,1,8
4,1104,32,Travel_Rarely,Research & Development,1,1,Life Sciences,134,4,Male,...,4,80,0,1,2,3,1,0,0,0


In [11]:
test['Label']=-1
print(len(test.columns))

32


In [12]:
train.head()

,ID,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Label
0,0,37,Travel_Rarely,Research & Development,1,4,Life Sciences,77,1,Male,...,80,1,7,2,4,7,5,0,7,0
1,1,54,Travel_Frequently,Research & Development,1,4,Life Sciences,1245,4,Female,...,80,1,33,2,1,5,4,1,4,0
2,2,34,Travel_Frequently,Research & Development,7,3,Life Sciences,147,1,Male,...,80,0,9,3,3,9,7,0,6,1
3,3,39,Travel_Rarely,Research & Development,1,1,Life Sciences,1026,4,Female,...,80,1,21,3,3,21,6,11,8,0
4,4,28,Travel_Frequently,Research & Development,1,3,Medical,1111,1,Male,...,80,2,1,2,3,1,0,0,0,1


In [13]:
data = train.append(test).reset_index(drop=True)
# data = pd.concat([train,test]).rese_index(drop=True)
print(data.head())

ID  Age     BusinessTravel              Department  DistanceFromHome  \
0   0   37      Travel_Rarely  Research & Development                 1   
1   1   54  Travel_Frequently  Research & Development                 1   
2   2   34  Travel_Frequently  Research & Development                 7   
3   3   39      Travel_Rarely  Research & Development                 1   
4   4   28  Travel_Frequently  Research & Development                 1   

   Education EducationField  EmployeeNumber  EnvironmentSatisfaction  Gender  \
0          4  Life Sciences              77                        1    Male   
1          4  Life Sciences            1245                        4  Female   
2          3  Life Sciences             147                        1    Male   
3          1  Life Sciences            1026                        4  Female   
4          3        Medical            1111                        1    Male   

   ...  StandardHours  StockOptionLevel TotalWorkingYears  \
0  ...    

In [14]:
path='./'
import os
import json
import gc
# os.system('pip install numba')
from numba import jit
#tqdm
# os.system('pip install tqdm')
from tqdm import tqdm_notebook
from tqdm import tqdm

#Integrated model
# os.system('pip install lightgbm')
import lightgbm as lgb
# os.system('pip install catboost==0.15.2')
import catboost as cbt
# os.system('pip install xgboost')
# import xgboost as xgb

#base import 
import numpy as np
import pandas as pd

# about sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler as std
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import f1_score
#about time
import time
import datetime 
from datetime import datetime, timedelta

#Garbage collection
import gc
# scipy
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
import scipy.spatial.distance as dist
#other
from collections import Counter 
from statistics import mode 
    #warning
import warnings
warnings.filterwarnings("ignore")
import json 
import math
from itertools import product
import ast 

In [15]:
cat_col = [i for i in data.select_dtypes(object).columns if i not in ['Label']]
for i in tqdm_notebook(cat_col):
    lbl = LabelEncoder()
#     data['count_' + i] = data.groupby([i])[i].transform('count')
    data[i] = lbl.fit_transform(data[i].astype(str))

In [16]:
data['Department'].head()

0    1
1    1
2    1
3    1
4    1
Name: Department, dtype: int32

In [17]:
data.head()

,ID,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Label
0,0,37,2,1,1,4,1,77,1,1,...,80,1,7,2,4,7,5,0,7,0
1,1,54,1,1,1,4,1,1245,4,0,...,80,1,33,2,1,5,4,1,4,0
2,2,34,1,1,7,3,1,147,1,1,...,80,0,9,3,3,9,7,0,6,1
3,3,39,2,1,1,1,1,1026,4,0,...,80,1,21,3,3,21,6,11,8,0
4,4,28,1,1,1,3,3,1111,1,1,...,80,2,1,2,3,1,0,0,0,1


In [18]:
feats = [i for i in data.columns if i not in ['ID','Label']]
feats

['Age',
 'BusinessTravel',
 'Department',
 'DistanceFromHome',
 'Education',
 'EducationField',
 'EmployeeNumber',
 'EnvironmentSatisfaction',
 'Gender',
 'JobInvolvement',
 'JobLevel',
 'JobRole',
 'JobSatisfaction',
 'MaritalStatus',
 'MonthlyIncome',
 'NumCompaniesWorked',
 'Over18',
 'OverTime',
 'PercentSalaryHike',
 'PerformanceRating',
 'RelationshipSatisfaction',
 'StandardHours',
 'StockOptionLevel',
 'TotalWorkingYears',
 'TrainingTimesLastYear',
 'WorkLifeBalance',
 'YearsAtCompany',
 'YearsInCurrentRole',
 'YearsSinceLastPromotion',
 'YearsWithCurrManager']

In [19]:
model = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=70, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary',metric= 'F1',
    subsample=0.95, colsample_bytree=0.95, subsample_freq=1,
    learning_rate=0.002, random_state=2017
    )
# train_x =data[data['Label']!=-1][feats]
# train_y =data[data['Label']!=-1]['Label']
# testx= data[data['Label']==-1][feats]
# # train_x, test_x, train_y, test_y = train_test_split(data1, label1, test_size=0.3, random_state=42)
# model.fit(train_x,train_y)
# test_pre = model.predict_proba(testx)[:,1]#, num_iteration=model.best_iteration

In [20]:
data.Label.value_counts()

 0    922
-1    350
 1    178
Name: Label, dtype: int64

In [21]:
data[data['Label']==-1][['ID']]

,ID
1100,1100
1101,1101
1102,1102
1103,1103
1104,1104
1105,1105
1106,1106
1107,1107
1108,1108
1109,1109


In [22]:
#5折交叉验证
from sklearn.model_selection import KFold
n_splits=5
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
# data.loc[data['Label']==2,'Label']=0
train_x = data[data['Label']!=-1][feats]
train_y = data[data['Label']!=-1]['Label']
res=data[data['Label']==-1][['ID']]

test_x= data[data['Label']==-1][feats]
res['pred'] = 0
for train_idx, val_idx in kfold.split(train_x):
    model.random_state = model.random_state + 1
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x.loc[val_idx]
    test_y1 = train_y.loc[val_idx]
    #,(vali_x,vali_y)
    model.fit(train_x1, train_y1,eval_set=[(train_x1, train_y1),(test_x1, test_y1)],eval_metric='auc',early_stopping_rounds=100)
    res['pred'] += model.predict_proba(test_x)[:,1]

res['pred'] = res['pred']/5
res['Label']=res['pred']
res.loc[res['Label']>=0.5,'Label']=1
res.loc[res['Label']<0.5,'Label']=0

's auc: 0.857639
[148]	training's auc: 0.943452	valid_1's auc: 0.857639
[149]	training's auc: 0.943595	valid_1's auc: 0.857488
[150]	training's auc: 0.943614	valid_1's auc: 0.85779
[151]	training's auc: 0.943729	valid_1's auc: 0.856884
[152]	training's auc: 0.943767	valid_1's auc: 0.857186
[153]	training's auc: 0.943796	valid_1's auc: 0.856582
[154]	training's auc: 0.943691	valid_1's auc: 0.857337
[155]	training's auc: 0.943929	valid_1's auc: 0.85628
[156]	training's auc: 0.944349	valid_1's auc: 0.856733
[157]	training's auc: 0.944492	valid_1's auc: 0.85628
[158]	training's auc: 0.94475	valid_1's auc: 0.856582
[159]	training's auc: 0.944702	valid_1's auc: 0.85628
[160]	training's auc: 0.944922	valid_1's auc: 0.857186
[161]	training's auc: 0.945027	valid_1's auc: 0.856582
[162]	training's auc: 0.94516	valid_1's auc: 0.857035
[163]	training's auc: 0.945275	valid_1's auc: 0.856884
[164]	training's auc: 0.945446	valid_1's auc: 0.857186
[165]	training's auc: 0.945714	valid_1's auc: 0.857337

In [23]:
res[['ID','Label']].to_csv('./output/lab_baseline_2.csv',index=False)

In [43]:
# 需要拿来融合的 概率文件
res[['ID','pred']].to_csv('./output/lgb_proba.csv',index=False)


In [47]:
from sklearn.ensemble import GradientBoostingClassifier
model=GradientBoostingClassifier()
testX=test_x.values
trainY=train_y.values
trainX=train_x.values
model.fit(trainX,trainY)
predictY=model.predict_proba(testX)[:,1]
rs =res[['ID']]
rs['pred'] = predictY
rs[['ID','pred']].to_csv('./output/gbdt_proba.csv',index=False)

In [48]:

rs.loc[rs['pred']>=0.5,'pred']=1
rs.loc[rs['pred']<0.5,'pred']=0
rs.columns=['ID','Label']
rs['Label']=res['Label'].astype(int)
rs[['ID','Label']].to_csv(path+'output/gbdt_baseline.csv', index=False)